In [1]:
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.device_count())  # Should return the number of GPUs
print(torch.cuda.get_device_name(0))  # Should display "NVIDIA GeForce RTX 4050"


True
1
NVIDIA GeForce RTX 4050 Laptop GPU


In [2]:
from datasets import load_dataset

dataset = load_dataset("Selinana/ECT-Gemini-summarization-dataset")
print(dataset)  # Shows available splits
print(dataset["train"].column_names)  # Shows columns in the training split


C:\Users\Dhir\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['split', 'input', 'label'],
        num_rows: 2425
    })
})
['split', 'input', 'label']


In [3]:
from datasets import load_dataset

dataset = load_dataset("Selinana/ECT-Gemini-summarization-dataset")
train_val_split = dataset["train"].train_test_split(test_size=0.2, seed=42)  # 80% train, 20% validation
train_data = train_val_split["train"]
val_data = train_val_split["test"]


In [4]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("t5-small")

def preprocess_data(examples):
    inputs = ["summarize: " + doc for doc in examples["input"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    labels = tokenizer(examples["label"], max_length=150, truncation=True)
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

tokenized_train = train_data.map(preprocess_data, batched=True)
tokenized_val = val_data.map(preprocess_data, batched=True)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [5]:
def preprocess_data(examples):
    # Tokenize the input text
    inputs = ["summarize: " + doc for doc in examples["input"]]
    model_inputs = tokenizer(inputs, max_length=512, padding="max_length", truncation=True)

    # Tokenize the labels (summary)
    labels = tokenizer(examples["label"], max_length=150, padding="max_length", truncation=True)

    # Add labels to model inputs
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs


In [6]:
tokenized_train = train_data.map(preprocess_data, batched=True, remove_columns=train_data.column_names)
tokenized_val = val_data.map(preprocess_data, batched=True, remove_columns=val_data.column_names)


In [7]:
from transformers import T5ForConditionalGeneration, TrainingArguments, Trainer

model = T5ForConditionalGeneration.from_pretrained("t5-small")

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
)

trainer.train()


C:\Users\Dhir\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\Dhir\AppData\Local\Temp\ipykernel_35500\314443307.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
  1%|▏         | 10/729 [00:03<03:26,  3.48it/s]

{'loss': 4.8924, 'grad_norm': 3.8521265983581543, 'learning_rate': 4.931412894375857e-05, 'epoch': 0.04}


  3%|▎         | 20/729 [00:06<03:18,  3.58it/s]

{'loss': 3.8396, 'grad_norm': 1.6995773315429688, 'learning_rate': 4.862825788751715e-05, 'epoch': 0.08}


  4%|▍         | 30/729 [00:09<03:14,  3.59it/s]

...

                                                 
100%|██████████| 729/729 [03:46<00:00,  4.08it/s]

{'eval_loss': 2.1858036518096924, 'eval_runtime': 4.7696, 'eval_samples_per_second': 101.685, 'eval_steps_per_second': 12.789, 'epoch': 3.0}


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].
100%|██████████| 729/729 [03:47<00:00,  3.21it/s]

{'train_runtime': 227.4089, 'train_samples_per_second': 25.593, 'train_steps_per_second': 3.206, 'train_loss': 2.6300475849209173, 'epoch': 3.0}


TrainOutput(global_step=729, training_loss=2.6300475849209173, metrics={'train_runtime': 227.4089, 'train_samples_per_second': 25.593, 'train_steps_per_second': 3.206, 'total_flos': 787689284567040.0, 'train_loss': 2.6300475849209173, 'epoch': 3.0})

In [8]:
from transformers import Trainer, DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,  # Add this
)



C:\Users\Dhir\AppData\Local\Temp\ipykernel_35500\2221422929.py:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [9]:
trainer.evaluate()


100%|██████████| 61/61 [00:04<00:00, 12.80it/s]


{'eval_loss': 2.1858036518096924,
 'eval_model_preparation_time': 0.002,
 'eval_runtime': 4.7967,
 'eval_samples_per_second': 101.11,
 'eval_steps_per_second': 12.717}

In [10]:

trainer.save_model("./t5-summarization-model")
tokenizer.save_pretrained("./t5-summarization-model")

('./t5-summarization-model\\tokenizer_config.json',
 './t5-summarization-model\\special_tokens_map.json',
 './t5-summarization-model\\spiece.model',
 './t5-summarization-model\\added_tokens.json')

In [11]:
import torch  # Import torch to use device handling
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the fine-tuned model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("./t5-summarization-model")
tokenizer = T5Tokenizer.from_pretrained("./t5-summarization-model")

# Move the model to the appropriate device (GPU or CPU)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

def summarize(text):
    inputs = tokenizer("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Move inputs to device
    outputs = model.generate(
        inputs["input_ids"],
        max_length=150,
        min_length=40,
        length_penalty=2.0,
        num_beams=4,
        early_stopping=True
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test the summarization function
test_input = "Your input text to summarize here."
print("Summary:", summarize(test_input))


Summary: Your input text to summarize here is a summary of your input text. The input text will be summarized in a graphical format, a graphical interface, and a graphical interface, a graphical interface.


In [12]:
def summarize(text):
    # Prepare the input by tokenizing the text
    inputs = tokenizer("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
    
    # Move inputs to the device (GPU) if available
    inputs = {key: value.to(model.device) for key, value in inputs.items()}

    # Generate the summary using the model
    outputs = model.generate(
        inputs['input_ids'],
        max_length=150, 
        min_length=40, 
        length_penalty=2.0, 
        num_beams=4, 
        early_stopping=True
    )

    # Decode and return the generated summary
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example input
test_input = "Ancient manuscripts also divided sentences into paragraphs with line breaks (newline) followed by an initial at the beginning of the next paragraph. An initial is an oversized capital letter, sometimes outdented beyond the margin of the text. This style can be seen, for example, in the original Old English manuscript of Beowulf. Outdenting is still used in English typography, though not commonly.[2] Modern English typography usually indicates a new paragraph by indenting the first line. This style can be seen in the (handwritten) United States Constitution from 1787. For additional ornamentation, a hedera leaf or other symbol can be added to the inter-paragraph white space, or put in the indentation space."
print("Summary:", summarize(test_input))


Summary: Original Old English manuscripts divided sentences into paragraphs with line breaks (newline) followed by an initial at the beginning of the next paragraph. An initial is an oversized capital letter, sometimes outdented beyond the margin of the text. This style can be seen in the (handwritten) United States Constitution from 1787.
